# Practice with basic RL algorithms
In this notebook I want to explore 3 basic algotithms of RL:


*   Multi-Armed Bandits (ε-greedy strategy)
*   Q-learning

## Multi-Armed Bandits
You have entered a casino where there are several slot machines (bandits), each of which has a different probability of winning.

You do not know which one is the winning one, so you have to experiment (try them all) and exploit (play the best one).

In [29]:
bandits = [0.1, 0.3, 0.5] #possibilities of winning with each authomat

In [30]:
import random

def pull(arm_idx):
  if random.random() <= bandits[arm_idx]:
    return 1
  return 0

In [31]:
win_counts = [0, 0, 0]
using_counts = [0, 0, 0]
exploring_prob = 0.1

for i in range(1000):
  arm_idx = 0
  if random.random() <= exploring_prob:
    arm_idx = random.randint(0, 2)
  else:
    arm_idx = win_counts.index(max(win_counts))

  win_counts[arm_idx] += pull(arm_idx)
  using_counts[arm_idx] += 1

print("win counts for each authomat: ", win_counts)
print("using counts of each authomat: ", using_counts)

win counts for each authomat:  [4, 289, 11]
using counts of each authomat:  [28, 945, 27]


## Q-learning

Q-learning is a model-free reinforcement learning algorithm that allows an agent to learn the optimal action policy in the environment through trial and error.



*   s (state) - the current situation of the agent in the environment
*   a (action) - what an agent can do
*   r (reward) - the numerical value that the agent receives for the action
*   Q(s, a) - expected long-term reward for action a in state s
*   policy - the agent's strategy for choosing an action in each state


In [32]:
# S - start, . - free, # - wall, G - goal
grid = [['S', '.', '.', '.'],
        ['#', '.', '#', '.'],
        ['.', '.', '.', '.'],
        ['.', '.', '#', 'G']]

start = (0, 0)
goal = (3, 3)

actions = ['up', 'down', 'right', 'left']

# Q(s, a)
Q = {}
for i in range(4):
    for j in range(4):
        if grid[i][j] != '#':
            Q[(i, j)] = {a: 0.0 for a in actions}

In [20]:
Q

{(0, 0): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (0, 1): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (0, 2): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (0, 3): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (1, 1): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (1, 3): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (2, 0): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (2, 1): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (2, 2): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (2, 3): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (3, 0): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (3, 1): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0},
 (3, 3): {'up': 0.0, 'down': 0.0, 'right': 0.0, 'left': 0.0}}

In [33]:
# hyperparameters
learning_rate = 0.2
consideretion_of_future = 0.9
research = 1
research_min = 0.1
epochs = 1000

In [38]:
import random

for i in range(epochs):
  cur_state = start

  while cur_state != goal:
    actions = Q[cur_state]
    if random.random() <= research:
      action_keys = list(actions.keys())
      a = random.choice(action_keys)
    else:
      a = max(actions, key=actions.get)

    if (a == 'up' and cur_state[0] > 0):
      next_state = (cur_state[0] - 1, cur_state[1])
    elif (a == 'down' and cur_state[0] < 3):
      next_state = (cur_state[0] + 1, cur_state[1])
    elif (a == 'right' and cur_state[1] < 3):
      next_state = (cur_state[0], cur_state[1] + 1)
    elif (a == 'left' and cur_state[1] > 0):
      next_state = (cur_state[0], cur_state[1] - 1)
    else:
      next_state = cur_state

    if grid[next_state[0]][next_state[1]] == '#':
        next_state = cur_state

    r = 0

    if grid[next_state[0]][next_state[1]] == '.':
      r = -0.01
    elif grid[next_state[0]][next_state[1]] == '#':
      r = -1
    elif grid[next_state[0]][next_state[1]] == 'G':
      r = 1

    new_actions = Q[next_state]
    new_best_a = max(new_actions, key=new_actions.get)
    new_best_a_value = new_actions[new_best_a]

    Q[cur_state][a] = Q[cur_state][a] + learning_rate * (r + consideretion_of_future * new_best_a_value - Q[cur_state][a])

    research = max(research * 0.99, research_min)
    cur_state = next_state

In [39]:
cur_state = start
road = []
road.append(cur_state)

while cur_state != goal:
  actions = Q[cur_state]
  a = max(actions, key=actions.get)

  if (a == 'up' and cur_state[0] > 0):
      next_state = (cur_state[0] - 1, cur_state[1])
  elif (a == 'down' and cur_state[0] < 3):
    next_state = (cur_state[0] + 1, cur_state[1])
  elif (a == 'right' and cur_state[1] < 3):
    next_state = (cur_state[0], cur_state[1] + 1)
  elif (a == 'left' and cur_state[1] > 0):
    next_state = (cur_state[0], cur_state[1] - 1)
  else:
    next_state = cur_state

  road.append(next_state)
  cur_state = next_state

print('Best road: ', road)

Best road:  [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3)]
